# Workshop 3: Data Pre-processing

COSC2671 Social Media and Network Analytics

Jeffrey Chan, RMIT University, 2023


This automatically reloads the client information if there are changes.

In [2]:
%load_ext autoreload
%autoreload 1
%aimport redditClient

Some necessary imports.

In [3]:
from redditClient import redditClient
import praw
import json
import functools
import datetime

Parameters used throughout this notebook.
Usually useful to have these parameters in one cell (and one location) and can edit them as needed.

In [4]:
# subreddit name we interested in getting the hot submissions
sSubredditName = 'NVDA_Stock'
# maximum number of hot submissions
hotLimit = 5

Construct Reddit client then print our name to test.

In [5]:
# construct Reddit client
client = redditClient()

# sanity check, you should see your own username printed out
print(client.user.me())

Comprehensive-End522


In [16]:
# @functools.lru_cache(maxsize=None)
def reddit_api_response(sSubredditName, hotLimit,start_date, end_date):  
    # specify which subreddit we are interested in
    subreddit = client.subreddit(sSubredditName)
    # Convert dates to timestamps
    start_timestamp = int(start_date.timestamp())
    end_timestamp = int(end_date.timestamp())

    data = []
    # print out the hot submisisons
    # subreddit.comments.replace_more(limit=None)
    # expand all the comments
    for submission in subreddit.top(limit=None):
        if start_timestamp <= submission.created_utc <= end_timestamp:
            for comment in submission.comments.list(): #only take last 10 comments
                comments=[]
                comment_data={
                    "author": comment.author.name if comment.author else "No Author",
                    "created": comment.created_utc,
                    "text": comment.body
                }
                comments.append(comment_data)
        
            # print title, but we can print other information as well
            post_data = {
            "title": submission.title,
            "author": submission.author.name if submission.author else "No Author",
            "score": submission.score,
            "comments":comments,
            "created":submission.created_utc
            }
            data.append(post_data)
        else:
            break
    print(data)
    final_data = {"submissions":data}
    with open("nvidia_stock_data.json", "w") as json_file:
        json.dump(final_data, json_file)
    print("Data saved to nvidia_stock_data.json")

In [20]:
start_date = datetime.datetime(2024, 8, 11, 0, 0, 0)  # Year, Month, Day, Hour, Minute, Second
end_date = datetime.datetime(2024, 8, 12, 0, 59, 0)
reddit_api_response(sSubredditName, hotLimit,start_date, end_date)

[]
Data saved to nvidia_stock_data.json
